In [1]:
import streamlit as st
import pandas as pd

In [2]:
from Features import world_constructions as WCf

In [3]:
from Features import search_from_luxury_brands as Sf

In [4]:
df = WCf.world_construction.init_luxury_gallery()

In [5]:
vectorestore = WCf.world_construction.init_chroma_db()

c:\Users\juand\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\juand\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\juand\anaconda3\Lib\site-packages\transformers\utils\generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\

In [6]:
description = df.iloc[0].Summary

In [7]:
entity = Sf.seacrh_from_luxury_brands(description=description, vectorstore=vectorestore)

In [14]:
answer = entity.generate_description_generalization()

In [17]:
answer

'This silk shirt features an eye-catching optical monogram print, designed with a regular fit and long sleeves with buttoned cuffs. It boasts a classic pointed collar, adding a touch of formality. The high-quality silk material provides a luxurious feel and fluid drape, making it suitable for all seasons due to its breathability and lightweight nature. The deep lavender and white colors are intricately interwoven to create a striking visual effect, with consistent pattern placement ensuring a cohesive look. The smooth, slightly glossy texture typical of silk is non-transparent, making it appropriate for both formal and semi-formal occasions. The shirt maintains a sleek and streamlined silhouette with no pockets, embellishments, or lapels. Its standard length typically falls just below the waistline and is finished with concealed buttons for a minimalist aesthetic.'